### SageMaker pretrain baichuan

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备requirements.txt

In [1]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 109.6 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.31.1
    Uninstalling botocore-1.31.1:
      Successfully uninstalled botocore-1.31.1
  Attempting uninstall: boto3
    Found existing installation: boto3 1.28.1
    Uninstalling boto3-1.28.1:
      Successfully uninstalled boto3-1.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.114 requires botocore==1.29.114, but you have botocore 1.31.4 which is incompatible.
awscli 1.27.114 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuro

In [2]:
import boto3
import sagemaker

account = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

/home/ec2-user/anaconda3/envs/amazonei_pytorch_latest_p37/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker-us-west-2-687912291502


In [3]:
!rm -rf ./LLaMA-Efficient-Tuning
!git clone -b baichuan-7b-rtx3090 https://github.com/hiyouga/LLaMA-Efficient-Tuning.git
!cp ./s5cmd ./LLaMA-Efficient-Tuning/

Cloning into 'LLaMA-Efficient-Tuning'...
remote: Enumerating objects: 1060, done.
remote: Counting objects: 100% (530/530), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 1060 (delta 445), reused 433 (delta 387), pack-reused 530
Receiving objects: 100% (1060/1060), 72.43 MiB | 16.25 MiB/s, done.
Resolving deltas: 100% (704/704), done.


## prepare docker images

In [ ]:
%%writefile Dockerfile
## You should change below region code to the region you used, here sample is use us-west-2
From 763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04 
#From pytorch/pytorch:1.5-cuda10.1-cudnn7-runtime

ENV LANG=C.UTF-8
ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE

#RUN pip install -U git+https://github.com/ssbuild/deep_training.git


In [ ]:
## You should change below region code to the region you used, here sample is use us-west-2
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-west-2.amazonaws.com

**Build image and push to ECR.**

In [5]:
## define repo name, should contain *sagemaker* in the name
repo_name = "sagemaker-baichuan_finetuning"

In [ ]:
%%script env repo_name=$repo_name bash

#!/usr/bin/env bash

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the image name. This will be used as the image on the local
# machine and combined with the account and region to form the repository name for ECR.
# The name of our algorithm
algorithm_name=${repo_name}

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

## pretrain baichuan

In [ ]:
##基础模型
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/LLM_baichuan_model/

In [10]:
####下载的baichuan model s3路径 ###########
model_s3_path="s3://sagemaker-us-west-2-687912291502/llm/models/LLM_baichuan_model"

In [11]:
## The image uri which is build and pushed above
image_uri = "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, repo_name)
image_uri

'687912291502.dkr.ecr.us-west-2.amazonaws.com/sagemaker-baichuan_finetuning:latest'

In [15]:
%%writefile ./LLaMA-Efficient-Tuning/requirements.txt
transformers>=4.29.1
datasets>=2.12.0
accelerate>=0.19.0
peft>=0.3.0
trl>=0.4.4
sentencepiece
jieba
rouge-chinese
nltk
gradio
mdtex2html
uvicorn
fastapi
sse-starlette
xformers

Overwriting ./LLaMA-Efficient-Tuning/requirements.txt


### step1：pretrain 

In [16]:
%%writefile ./LLaMA-Efficient-Tuning/pretrain.sh
#!/bin/bash

chmod +x ./s5cmd
pip install -U -r requirements.txt

#./s5cmd sync $MODEL_S3_PATH/* /tmp/baichun-7b/


export CUDA_VISIBLE_DEVICES=0

CUDA_VISIBLE_DEVICES=0 python src/train_pt.py \
    --model_name_or_path "fireballoon/baichuan-vicuna-7b" \
    --do_train \
    --dataset wiki_demo \
    --finetuning_type lora \
    --output_dir /tmp/output/ \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 5e-5 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16

./s5cmd sync /tmp/ouput/ $MODEL_S3_PATH/models/baichuan_pretrain/output/$(date +%Y-%m-%d-%H-%M-%S)/

Overwriting ./LLaMA-Efficient-Tuning/pretrain.sh


### step2：sft (人工标注/回答语料）

In [ ]:
%%writefile ./LLaMA-Efficient-Tuning/train_fine_tune.sh
chmod +x ./s5cmd
pip install -U -r requirements.txt

./s5cmd sync $MODEL_S3_PATH/models/baichuan_pretrain/output/$(date +%Y-%m-%d-%H-%M-%S)/* /tmp/baichun-pretain/


export CUDA_VISIBLE_DEVICES=0,1,2,3

accelerate launch --multi_gpu src/train_sft.py \
    --model_name_or_path /tmp/baichun-pretain/ \
    --do_train \
    --dataset alpaca_zh \
    --finetuning_type lora \
    --output_dir /tmp/ouput/ \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 5e-5 \
    --num_train_epochs 1.0 \
    --plot_loss \
    --fp16

./s5cmd sync /tmp/ouput/ $MODEL_S3_PATH/models/baichuan_finetuning/output/$(date +%Y-%m-%d-%H-%M-%S)/

### step3: train reward （上一步的model输出结果，由人工标注排序语料）

In [ ]:
%%writefile ./LLaMA-Efficient-Tuning/train_reward.sh
chmod +x ./s5cmd
pip install -U -r requirements.txt

./s5cmd sync $MODEL_S3_PATH/models/baichuan_finetuning/output/$(date +%Y-%m-%d-%H-%M-%S)/* /tmp/baichun-finetuning/



CUDA_VISIBLE_DEVICES=0 python src/train_rm.py \
    --model_name_or_path /tmp/baichun-finetuning/ \
    --do_train \
    --dataset comparison_gpt4_en \
    --finetuning_type lora \
    --output_dir /tmp/output/ \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 1e-5 \
    --num_train_epochs 1.0 \
    --plot_loss \
    --fp16

./s5cmd sync /tmp/ouput/ $MODEL_S3_PATH/models/baichuan_reward/output/$(date +%Y-%m-%d-%H-%M-%S)/

### step4: PPO training

In [ ]:
%%writefile ./LLaMA-Efficient-Tuning/train_ppo.sh
chmod +x ./s5cmd
pip install -U -r requirements.txt

./s5cmd sync $MODEL_S3_PATH/models/baichuan_reward/output/$(date +%Y-%m-%d-%H-%M-%S)/* /tmp/baichuan_reward/
./s5cmd sync $MODEL_S3_PATH/models/baichuan_finetuning/output/$(date +%Y-%m-%d-%H-%M-%S)/* /tmp/baichuan_finetuning/

CUDA_VISIBLE_DEVICES=0 python src/train_ppo.py \
    --model_name_or_path /tmp/baichuan_finetuning/ \
    --do_train \
    --dataset alpaca_gpt4_en \
    --finetuning_type lora \
    --checkpoint_dir /tmp/baichuan_finetuning/ \
    --reward_model /tmp/baichuan_reward/ \
    --output_dir /tmp/output/ \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 1e-5 \
    --num_train_epochs 1.0 \
    --resume_lora_training False \
    --plot_loss

./s5cmd sync /tmp/ouput/ $MODEL_S3_PATH/models/baichuan_ppo/output/$(date +%Y-%m-%d-%H-%M-%S)/

In [ ]:
import time
from sagemaker.estimator import Estimator

environment = {
              'MODEL_S3_PATH': model_s3_path # The bucket to store pretrained model and fine-tune model
}

base_job_name = 'baichuan-finetuning'

instance_type = 'ml.g5.24xlarge'

estimator = Estimator(role=role,
                      entry_point='pretrain.sh',
                      source_dir='./LLaMA-Efficient-Tuning/',
                      base_job_name=base_job_name,
                      instance_count=1,
                      instance_type=instance_type,
                      image_uri=image_uri,
                      environment=environment,
                      disable_profiler=True,
                      debugger_hook_config=False,
                      max_run=24*60*60*2)

estimator.fit()

Using provided s3_resource


INFO:sagemaker:Creating training-job with name: baichuan-finetuning-2023-07-18-13-18-32-090


2023-07-18 13:18:49 Starting - Starting the training job...